In [ ]:
import holoviews as hv
import numpy as np
import panel as pn
import param

from scipy.special import comb

hv.extension('bokeh')

In [ ]:
class SplineEditor(param.Parameterized):

    control_size = param.Integer(default=5)
    line_width = param.Integer(default=2)
    
    def __init__(self, **params):
        super().__init__(**params)
        self.control_points = {}
        self.quad = hv.Points([])
        self._quad_stream = hv.streams.PointDraw(num_objects=4, source=self.quad, data={'x': [], 'y': []})
        self._control_edit = hv.streams.PointDraw(num_objects=1, rename={'data': 'controls'})
        self.spline_dmap = hv.DynamicMap(self._spline, streams=[self._quad_stream, self._control_edit])
        self.control_dmap = hv.DynamicMap(self._controls, streams=[self._quad_stream])
        self._control_edit.source = self.control_dmap
        self._control_edit.add_subscriber(self._update_controls)
    
    def _update_controls(self, controls):
        xs, ys = controls['x'], controls['y']
        self.control_points = {i: (x, y) for i, (x, y) in enumerate(zip(xs, ys))}   
    
    @classmethod
    def bernstein_poly(cls, i, n, t):
        """
         The Bernstein polynomial of n, i as a function of t
        """
        return comb(n, i) * ( t**(n-i) ) * (1 - t)**i
    
    @classmethod
    def bezier_curve(cls, points, nTimes=10):
        nPoints = len(points)
        xPoints = np.array([p[0] for p in points])
        yPoints = np.array([p[1] for p in points])
        t = np.linspace(0.0, 1.0, nTimes)
        polynomial_array = np.array([
            cls.bernstein_poly(i, nPoints-1, t)
            for i in range(0, nPoints)
        ])
        print(points)
        xvals = np.dot(xPoints, polynomial_array)
        yvals = np.dot(yPoints, polynomial_array)
        return xvals, yvals
    
    def _spline(self, data, controls):
        xs, ys = data['x'], data['y']
        lines = []
        for i in range(len(xs)-(1 if len(xs) < 4 else 0)):
            s, e = i, (i+1)%4
            controls = [(xs[s], ys[s]), (xs[e], ys[e])]
            if i in self.control_points:
                controls.insert(1, self.control_points[i])
            lines.append(self.bezier_curve(controls))
        return hv.Path(lines)
    
    def _controls(self, data):
        xs, ys = data['x'], data['y']
        controls = []
        for i in range(len(xs)-(1 if len(xs) < 4 else 0)):
            if i in self.control_points:
                controls.append(self.control_points[i])
            else:
                s, e = i, (i+1)%4
                controls.append(((xs[s]+xs[e])/2., (ys[s]+ys[e])/2.))
                self.control_points[i] = controls[-1]
        return hv.Points(controls)
    
    def view(self):
        return pn.Row(
            self.param,
            self.quad.apply.opts(size=self.param.control_size) *
            self.spline_dmap.apply.opts(line_width=self.param.line_width) *
            self.control_dmap.apply.opts(size=self.param.control_size)
        )

In [ ]:
editor = SplineEditor()
editor.view()